In [10]:
import numpy as np
import pandas as pd
import pickle

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
import nn_models

import sys, os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
sys.path.insert(1, os.path.join(sys.path[0], '..')) 
import custom_utils


def init_dirs(config_file):
    out_dir = custom_utils.create_out_dir(config_file)
    out_dir = '../' + out_dir	
    ml_data_dir = out_dir + '/ml_data'

    return out_dir, ml_data_dir


def read_data():
    print('> Loading data (top ' + str(100 * float(top_ratio)) + ' %) ...')
    top_ratio_str = '.top_' + str(top_ratio)

    print('Reading training data...')
    pkl_in = open(ml_data_dir + '/train' + top_ratio_str + '.pkl', 'rb')
    train_dict = pickle.load(pkl_in)
    pkl_in.close()

    print('Reading validation data...')
    pkl_in = open(ml_data_dir + '/validation' + top_ratio_str + '.pkl', 'rb')
    validation_dict = pickle.load(pkl_in)
    pkl_in.close()

    print('Reading test data...')
    pkl_in = open(ml_data_dir + '/test' + top_ratio_str + '.pkl', 'rb')
    test_dict = pickle.load(pkl_in)
    pkl_in.close()

    return train_dict, validation_dict, test_dict


def inspect_input_data(train_dict, validation_dict, test_dict):

    print('\n> Train:')
    print(train_dict['X'].shape)
    print(train_dict[y].shape)
    print(train_dict['seqs'].shape)

    print('\n> Validation:')
    print(validation_dict['X'].shape)
    print(validation_dict[y].shape)
    print(validation_dict['seqs'].shape)

    print('\n> Test:')
    print(test_dict['X'].shape)
    print(test_dict[y].shape)
    print(test_dict['seqs'].shape)



def compile_and_train_model(model, epochs=40):

    if regression:
        model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    else:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['cosine'])

    checkpoint_name = 'gwrvis_best_model.hdf5'
    checkpointer = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    model.fit(train_dict['seqs'], train_dict[y], batch_size=2048, epochs=epochs, 
          shuffle=True,
          validation_data=(validation_dict['seqs'], validation_dict[y]), 
          callbacks=[checkpointer,earlystopper])

    return model


def test_and_evaluate_model(model):
    test_results = model.evaluate(test_dict['seqs'], test_dict[y]) 
    print(test_results)

    preds = model.predict(test_dict['seqs'])
    decision_thres = 0.5 # for classification
    if regression:
        decision_thres = 0 # 0 is the natural border between tolerant and intolerant gwRVIS values

    preds[preds >= decision_thres] = 1
    preds[preds < decision_thres] = 0

    preds_flat = preds.flatten()
    test_flat = test_dict[y].flatten()

    print(accuracy_score(test_flat, preds_flat))
    print(confusion_matrix(test_flat, preds_flat))

    roc_auc = roc_auc_score(test_flat, preds_flat)
    print('ROC AUC:', roc_auc)

In [15]:
if __name__ == '__main__':

    config_file = '../config.yaml' #sys.argv[1]
    top_ratio = 0.001 #sys.argv[2] 	#default: 0.01 -- look at top 1% of intolerant/tolerant windows

    regression=False
    if regression:
        y = 'gwrvis'   # continuous value
    else:
        y = 'y'  # 1/0 annotation

    config_params = custom_utils.get_config_params(config_file)
    win_len = config_params['win_len']

    # init out dir structure
    out_dir, ml_data_dir = init_dirs(config_file)

    # read train, validation, test data
    train_dict, validation_dict, test_dict = read_data()

    # print input data shapes to check for consistency
    inspect_input_data(train_dict, validation_dict, test_dict)

    model = nn_models.cnn_1_conv_2_fcc(regression=regression) 
#     model = nn_models.cnn_rnn_1_conv_1_lstm(regression=regression)
    print(model.summary())

    model = compile_and_train_model(model, epochs=40)

> Loading data (top 0.1 %) ...
Reading training data...
Reading validation data...
Reading test data...

> Train:
(1240, 5)
(1240, 2)
(1240, 3000, 4)

> Validation:
(64, 5)
(64, 2)
(64, 3000, 4)

> Test:
(310, 5)
(310, 2)
(310, 3000, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 2971, 16)          1936      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 198, 16)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 198, 16)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3168)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               405632    
_____________________________________

In [17]:
preds = model.predict(test_dict['seqs'])
print(preds)

decision_thres = 0.5 # for classification
if regression:
    decision_thres = 0 # 0 is the natural border between tolerant and intolerant gwRVIS values

preds[preds >= decision_thres] = 1
preds[preds < decision_thres] = 0
print(preds)

[[0.6666713  0.3333287 ]
 [0.6330297  0.36697033]
 [0.6516117  0.3483883 ]
 [0.6495804  0.3504196 ]
 [0.70192283 0.29807717]
 [0.6854742  0.31452578]
 [0.67706525 0.32293475]
 [0.69077575 0.3092243 ]
 [0.6583584  0.34164155]
 [0.64561146 0.35438856]
 [0.71210897 0.28789103]
 [0.6467439  0.35325608]
 [0.6849443  0.3150557 ]
 [0.6910312  0.3089688 ]
 [0.63427615 0.3657238 ]
 [0.6569383  0.34306175]
 [0.65711826 0.34288174]
 [0.69592595 0.30407408]
 [0.6324239  0.3675761 ]
 [0.66454005 0.33545995]
 [0.6633434  0.3366566 ]
 [0.6832921  0.3167079 ]
 [0.6547281  0.34527186]
 [0.67860126 0.32139876]
 [0.67624205 0.32375792]
 [0.67345697 0.32654306]
 [0.6442981  0.35570183]
 [0.68952787 0.3104721 ]
 [0.63235587 0.36764407]
 [0.63992333 0.36007664]
 [0.6738321  0.3261679 ]
 [0.6450755  0.35492456]
 [0.6224532  0.37754676]
 [0.6539317  0.3460683 ]
 [0.6467276  0.35327238]
 [0.6616685  0.33833155]
 [0.65128213 0.34871784]
 [0.66833425 0.33166578]
 [0.6456256  0.35437438]
 [0.6420474  0.3579526 ]


In [13]:
preds_flat = np.argmax(preds, axis=1)
test_flat = np.argmax(test_dict[y], axis=1)

print(preds_flat)
# print(accuracy_score(test_flat, preds_flat))
# print(confusion_matrix(test_flat, preds_flat))

# roc_auc = roc_auc_score(test_flat, preds_flat)
# print('ROC AUC:', roc_auc)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [14]:
print(test_flat[ test_flat == 0 ].shape)
print(test_flat[ test_flat == 1 ].shape)

print(preds_flat[ preds_flat == 0 ].shape)
print(preds_flat[ preds_flat == 1 ].shape)

print('Accuracy:', accuracy_score(test_flat, preds_flat))
print(confusion_matrix(test_flat, preds_flat))

roc_auc = roc_auc_score(test_flat, preds_flat)
print('ROC AUC:', roc_auc)

(160,)
(150,)
(286,)
(24,)
Accuracy: 0.5806451612903226
[[158   2]
 [128  22]]
ROC AUC: 0.5670833333333334


In [24]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

def plot_roc_curve(test_flat, preds_flat, make_plot=True):

    fpr, tpr, _ = roc_curve(test_flat, preds_flat)
    roc_auc = roc_auc_score(test_flat, preds_flat)

    if make_plot:
        f = plt.figure(figsize=(6, 6))
        _ = plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
        _ = plt.plot([0, 1], [0, 1], '--', linewidth=0.5)  # random predictions curve

        _ = plt.xlim([0.0, 1.0])
        _ = plt.ylim([0.0, 1.0])
        _ = plt.title('\nROC (area = %0.3f)' % roc_auc)
        _ = plt.xlabel('False Positive Rate (1 — Specificity)')
        _ = plt.ylabel('True Positive Rate (Sensitivity)')
        plt.grid(True)
        plt.show()

        f.savefig("ROC_curve.pdf", bbox_inches='tight')

    return fpr, tpr
    
plot_roc_curve(test_flat, preds_flat)

/home/djifos/.conda/envs/tf/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


(array([0.   , 0.775, 1.   ]), array([0.        , 0.89333333, 1.        ]))